TF-IDF

In [1]:
import numpy as np 
import pandas as pd 
import re

In [2]:
from datetime import datetime
startTimeScript = datetime.now()

In [3]:
normal_tweets_text_sample_df=pd.read_excel('normal_tweets_text_sample.xlsx')
depressive_tweets_text_sample_df=pd.read_excel('depressive_tweets_text_sample.xlsx')
also_depressive_tweets_text_sample_df=pd.read_excel('normal_tweets_text_sample_0only.xlsx')
tweets_text_sample_df=pd.concat([normal_tweets_text_sample_df,depressive_tweets_text_sample_df,also_depressive_tweets_text_sample_df])
tweets_text_sample_df=tweets_text_sample_df.iloc[:,1:6]
tweets_text_sample_df = tweets_text_sample_df.dropna(axis=0,subset = ["normalized_text"])
tweets_text_sample_df


,id,text,lable,normalized_text
0,1680093728,@Dreameress1 Hello!!! Hows it going?,0,hello going
1,2053040367,is on the train to london,0,train london
2,2060651098,@TheLadyJane hey i wrote you a message today ...,0,hey wrot mess today kind gam ar ref playin
3,2059350074,"@SuperRecords goodnight , im off to bed also ...",0,goodnight im bed also nic talk d
4,1960820806,@SilverSteer I am always wearing turquoise jew...,0,alway wear turquo jewel fav
...,...,...,...,...
7995,1833084157,I can't download K-lite codec pack,1,not download klite codec pack
7996,2214628609,@britishxo and u didnt come see me !,1,u didnt com see
7997,2245101277,looking forward to a great weekend of revising...,1,look forward gre weekend rev gcse monday pah ...
7998,1997866869,Heidi and spencer are michigamua-esque,1,heid spent ar michigamuaesque


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
text_tv_fit = tv.fit_transform(tweets_text_sample_df['normalized_text'].values.astype('U'))
text_tv_fit_features = tv.get_feature_names()
text_tv_fit_matrix = text_tv_fit.toarray()
text_tv_fit_matrix

D:\Anaconda3\envs\tf2\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
len(text_tv_fit_features)

16025

**SVM**+**NB**

In [6]:
from sklearn.model_selection import train_test_split
np.random.seed(1234)
X=text_tv_fit_matrix
y=pd.DataFrame(tweets_text_sample_df[['text','normalized_text','lable']])
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)

svm

In [ ]:
from sklearn import svm
clf_svm = svm.SVC(C=0.1)
clf_svm.fit(X_train, y_train['lable'])

In [ ]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

In [ ]:
print("accuravy_train：",clf_svm.score(X_train,y_train['lable']))
y_pred = clf_svm.predict(X_test)

In [ ]:
svm_result=y_test[:].reset_index()
svm_result['pred']=y_pred
svm_result=svm_result[svm_result['lable']!=svm_result['pred']]
svm_result.to_excel('svm_wrong_result.xlsx',index=False)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test['lable'], y_pred))

print("accuracy:", metrics.accuracy_score(y_test['lable'], y_pred))
print("precision:", metrics.precision_score(y_test['lable'], y_pred, average='macro'))
print("recall:", metrics.recall_score(y_test['lable'], y_pred, average='macro'))
print("f1:", metrics.f1_score(y_test['lable'], y_pred, average='weighted'))


In [ ]:
y_score = clf_svm.decision_function(X_test)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, metrics, model_selection, svm
sns.set_style('whitegrid')

display = metrics.plot_roc_curve(clf_svm, X_test, y_test['lable'])
print('type(display):',type(display))
plt.plot([0, 1], [0, 1], color='navy',linestyle='--')
plt.savefig("ROC_svm.png")

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

f,ax=plt.subplots()
CM= confusion_matrix(y_test['lable'], y_pred)
print(CM)
sns.heatmap(CM,ax=ax, cmap="YlGn")
for i in range(2):
    for j in range(2):
        text = ax.text(j+0.5, i+0.5, CM[i, j],
                       ha="center", va="center", color="k")

ax.text(0.5, 0.5, CM[0, 0],ha="center", va="center", color="w")

ax.set_title('Confusion Matrix') 
ax.set_xlabel('Predicted Label') 
ax.set_ylabel('True Label')
plt.savefig("confusion matrix_svm.png")

NB

In [7]:
from sklearn.naive_bayes import MultinomialNB

clf_nb = MultinomialNB()
clf_nb.fit(X_train, y_train['lable'])

MultinomialNB()

In [8]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:00:10.263504


In [ ]:
print("accuravy_train：",clf_nb.score(X_train,y_train['lable']))
y_pred = clf_nb.predict(X_test)

In [ ]:
nb_result=y_test[:].reset_index()
nb_result['pred']=y_pred
nb_result=nb_result[nb_result['lable']!=nb_result['pred']]
nb_result.to_excel('nb_wrong_result.xlsx',index=False)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test['lable'], y_pred))

print("accuracy:", metrics.accuracy_score(y_test['lable'], y_pred))
print("precision:", metrics.precision_score(y_test['lable'], y_pred, average='macro'))
print("recall:", metrics.recall_score(y_test['lable'], y_pred, average='macro'))
print("f1:", metrics.f1_score(y_test['lable'], y_pred, average='weighted'))


In [ ]:
y_score = clf_nb.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, metrics, model_selection, svm
sns.set_style('whitegrid')

display = metrics.plot_roc_curve(clf_nb, X_test, y_test['lable'])
print('type(display):',type(display))
plt.plot([0, 1], [0, 1], color='navy',linestyle='--')
plt.savefig("ROC_NB.png")

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

sns.set()
f,ax=plt.subplots()
CM= confusion_matrix(y_test['lable'], y_pred)
print(CM)
sns.heatmap(CM,ax=ax, cmap="YlGn")
for i in range(2):
    for j in range(2):
        text = ax.text(j+0.5, i+0.5, CM[i, j],
                       ha="center", va="center", color="k")

ax.text(0.5, 0.5, CM[0, 0],ha="center", va="center", color="w")

ax.set_title('Confusion Matrix') 
ax.set_xlabel('Predicted Label') 
ax.set_ylabel('True Label')
plt.savefig("confusion matrix_NB.png")